In [1]:
import numpy as np
import pandas as pd
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')

In [2]:
def request(rqdict):
    t0 = time.time()

    c = cdsapi.Client()
    r = c.retrieve(
        'insitu-comprehensive-upper-air-observation-network',rqdict)
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
#     z = zipfile.ZipFile('download.zip')
#     z.extractall(path='./test_requests/')
#     z.close()
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./test/')
    z.close()
    
    files = glob.glob('./test/*.nc')
    if len(files) > 0:
        ds = xarray.open_dataset(files[0])            
        to_be_removed = ['trajectory_index', 'trajectory']
        for ivar in list(ds.variables):
            if 'string' in ivar:
                to_be_removed.append(ivar)
            
            if 'trajectory' in ds[ivar].dims and ivar not in list(ds.coords):
                report_id = ds[ivar].astype(object).sum(axis=1).astype(str)
                ds = ds.drop_vars(ivar)
                ds[ivar] = ('obs', report_id.values[ds.trajectory_index.values])  # todo obs ???
                
            if ds[ivar].ndim > 1:
                tmp= ds[ivar].astype(object).sum(axis=1).astype(str)
                ds = ds.drop_vars(ivar)
                idim = tmp.dims[0]
                ds[ivar] = (idim, tmp)

        ds = ds.drop_vars(to_be_removed)
        data = ds.to_dataframe()
    else:
        files = glob.glob('./test/*.csv')
        data = pd.read_csv(files[0])
    
    os.remove('download.zip')
    try:
       shutil.rmtree('./test/')
    except:
       print('Error while deleting directory')
    
    return data

In [3]:
def test_statid():
    log = []
    log.append('\n\n---STATID---')
    cachworkaround = np.random.random_sample()
    for s in [None, 'all', 11035, '11035', '0-20000-0-11035', '0-20100-0-00701','11*', '1103*', '0-20100*', ['11035', 10393], ['0-20000-0-11035', '0-20000-0-10393']]:
        print('statid - testing: ', s)
        rd={
            'variable':'air_temperature',
            'statid':s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['110350', '1103', '0-20000-0-1103', '0-20000-0-110350']:
        print('statid - testing: ', s)
        rd={
            'variable':'air_temperature',
            'statid':s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        }
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('statid - testing: finished')
    return log

In [4]:
def test_variable():
    log = []
    log.append('\n\n---VARIABLE---')
    cachworkaround = np.random.random_sample()
    for s in ['air_temperature', 'dew_point_temperature', 'eastward_wind_speed', 'northward_wind_speed', 'relative_humidity', 'specific_humidity', 'wind_from_direction', 'wind_speed']:
        print('variable - testing: ', s)
        rd={
            'variable': s,
            'statid': '0-20000-0-11035',
            'date': '20150101-20150131',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['temperature', 'dew_point', 'wind', 'humidity', 'sonde_type']:
        print('variable - testing: ', s)
        rd={
            'variable': s,
            'statid': '0-20000-0-11035',
            'date': '20150101-20150131',
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('variable - testing: finished')
    return log

In [5]:
def test_optional():
    log = []
    log.append('\n\n---OPTIONAL---')
    cachworkaround = np.random.random_sample()
    for s in ['sonde_type', 'bias_estimate', 'obs_minus_an', 'obs_minus_bg']:
        print('optional - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': ['0-20000-0-11035', '0-20100-0-00701'],
            'date': '19781001-19781031',
            'optional': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ ' - ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['sonde', 'air_temperature', 'dew_point_temperature']:
        print('optional - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': ['0-20000-0-11035', '0-20100-0-00701'],
            'date': '19781001-19781031',
            'optional': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('optional - testing: finished')
    return log

In [6]:
def test_area():
    log = []
    log.append('\n\n---AREA---')
    cachworkaround = np.random.random_sample()
    for s in [[80,140,-40,359], [50,0,40,20], [49,-180,48,180], [49,0,48,360], [90, -180, -90, 180]]:
        print('area - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'area': s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in [[80,140,70], [80,140,70,150,0],[0,0,0,0], [40,20,50,0], [49,-180,48,360]]:
        print('area - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'area': s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('area - testing: finished')
    return log

In [7]:
def test_country():
    log = []
    log.append('\n\n---COUNTRY---')
    cachworkaround = np.random.random_sample()
    for s in ['GLOBE', 'ALL', 'NOR', ["NOR","DEU"]]:
        print('country - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'country': s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['NORWAY', 'nor',]:
        print('country - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'country': s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('country - testing: finished')
    return log

In [8]:
def test_date(): 
    log = []
    log.append('\n\n---DATE---')
    cachworkaround = np.random.random_sample()
    for s in [None, '20000231', '20000101', '20000101-20000201', ['20000101', '20000103', '20000106'], ['20000102', '20000101']]:
        print('date - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['2000011', '20000100', '00000000', '20000132', '20001301', '20000201-20000101', '20000101/20000201']:
        print('date - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('date - testing: finished')
    return log

In [9]:
def test_period(): 
    log = []
    log.append('\n\n---PERIOD---')
    # not working: ['1979-01-01/1980-08-16']
    cachworkaround = np.random.random_sample()
    for s in [None, '1979-01-01/1980-08-16']:
        print('period - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'period': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['1980-01-02/1980-01-01']:
        print('period - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'period': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('period - testing: finished')
    return log

In [10]:
def test_pressure_level():
    log = []
    log.append('\n\n---PRESSURE LEVEL---')
    cachworkaround = np.random.random_sample()
    for s in [100, '100', [100, 200], ['100', '200']]:
        print('pressure_level - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20100101',
            'pressure_level': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
        
    for s in ['0', '2000']:
        print('pressure_level - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20100101',
            'pressure_level': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
        
    print('pressure_level - testing: finished')
    return log

In [11]:
def test_time():
    log = []
    log.append('\n\n---TIME---')
    cachworkaround = np.random.random_sample()
    for s in [[22,23,0,1,2,3], ["22","23","00","01","02","03"], '', '0', '00', ["00"], [0,12], ["0","12"]]:
        print('time - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20100101-20100201',
            'time': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['24', '25', '12.5', ]:
        print('time - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20100101-20100201',
            'time': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('time - testing: finished')
    return log

In [12]:
def test_format():
    log = []
    log.append('\n\n---FORMAT---')
    cachworkaround = np.random.random_sample()
    for s in [None, 'csv', 'nc']:
        print('format - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20100101-20100201',
            'format': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['netcdf', 'csb', 'txt']:
        print('format - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20100101-20100201',
            'format': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('format - testing: finished')
    return log

In [13]:
def test_ymd():
    log = []
    # if one of them is None -> no date selected = the whole dataset - change?
    cachworkaround = np.random.random_sample()
    log.append('\n\n---YEAR---')
    for s in [None, 2000, '2000', ['2000', '2001'], [2000, 2001]]:
        print('y - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': s,
            'month': '01',
            'day':'01',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in [0, '20000', '-2000']:
        print('y - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': s,
            'month': '01',
            'day':'01',
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    log.append('\n\n---MONTH---')
    for s in [None, 1, '1', '01', ['01', '02'], ['1', '2'], [1, 2]]:
        print('m - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': '2000',
            'month': s,
            'day':'01',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
    
    for s in ['00', 0, '13']:
        print('m - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': '2000',
            'month': s,
            'day':'01',
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    log.append('\n\n---DAY---')
    for s in ['01', '31', ['01', '02'], ['1', '2'], [1, 2]]:
        print('d - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': '2000',
            'month': '01',
            'day':s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
    
    for s in ['00', 0, '32']:
        print('d - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': '2000',
            'month': '01',
            'day':s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('ymd - testing: finished')
    return log

In [14]:
def test_size():
    log = []
    log.append('\n\n---SIZE---')
    cachworkaround = np.random.random_sample()
    s = 'air_temperature 11035'
    print('size - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'statid': '11035',
        'cachworkaround': cachworkaround,
    } 
    try:
        display(request(rd))
        log.append('\n1 request successful: ' + str(s))
    except Exception as e:
        log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
    
    s = 'air_temperature 10393'
    print('size - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'statid': '10393',
        'cachworkaround': cachworkaround,
    } 
    try:
        display(request(rd))
        log.append('\n1 request successful: ' + str(s))
    except Exception as e:
        log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
        
    s = 'air_temperature 1 month all stations'
    print('size - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'date': '20100101-20100201',
        'cachworkaround': cachworkaround,
    } 
    try:
        display(request(rd))
        log.append('\n1 request successful: ' + str(s))
    except Exception as e:
        log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
            
    s = 'air_temperature everything'
    print('size - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'cachworkaround': cachworkaround,
    }
    try: 
        display(request(rd))
        log.append('\n0 request falsely successful: ' + str(s))
    except Exception as e:
        print(e)
        log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
    
    s = 'all variables 1 month all stations'
    print('size - testing: ', s)
    rd={
        'variable': ['air_temperature', 'dew_point_temperature', 'eastward_wind_speed', 'northward_wind_speed', 'relative_humidity', 'specific_humidity', 'wind_from_direction', 'wind_speed'],
        'date': '20100101-20100201',
        'cachworkaround': cachworkaround,
    }
    try: 
        display(request(rd))
        log.append('\n0 request falsely successful: ' + str(s))
    except Exception as e:
        print(e)
        log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('size - testing: finished')
    return log

In [15]:
def server_test():
    log = []
    log.append('Testing Server Requests \n')
    log.append('0 = failed | 1 = succeeded \n')
    a = !git rev-parse HEAD
    log.append('GIT revision: ' + str(a) + ' \n')
    log.append(str(date.today())+ ' \n')
    try: log.extend(test_statid())
    except: log.append('\nstatid - test: failed \n')
    try: log.extend(test_variable())
    except: log.append('\nvariable - test: failed \n')
    try: log.extend(test_optional())
    except: log.append('\noptional - test: failed \n')
    try: log.extend(test_area())
    except: log.append('\narea - test: failed \n')
    try: log.extend(test_country())
    except: log.append('\ncountry - test: failed \n')
    try: log.extend(test_date())
    except: log.append('\ndate - test: failed \n')
    try: log.extend(test_period())
    except: log.append('\nperiod - test: failed \n')
    try: log.extend(test_pressure_level())
    except: log.append('\npressure_level - test: failed \n')
    try: log.extend(test_time())
    except: log.append('\ntime - test: failed \n')
    try: log.extend(test_format())
    except: log.append('\nformat - test: failed \n')
    try: log.extend(test_ymd())
    except: log.append('\nymd - test: failed \n')
    try: log.extend(test_size())
    except: log.append('\nsize - test: failed \n')
        
    writelog=open(str(date.today()) + '_server_test_log.txt','w')
    writelog.writelines(log)
    writelog.close()

In [16]:
server_test()

2020-12-11 08:28:54,769 INFO Welcome to the CDS
2020-12-11 08:28:54,770 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network


statid - testing:  None


2020-12-11 08:28:55,047 INFO Request is queued
2020-12-11 08:28:56,082 INFO Request is running
2020-12-11 08:29:08,416 INFO Request is completed
2020-12-11 08:29:08,417 INFO Downloading http://136.156.132.204/cache-compute-0001/cache/data1/adaptor.comprehensive_upper_air.retrieve-1607671744.8848436-3111-3-70f6745d-e29b-4a5d-9eba-f36a67a17610.zip to download.zip (49.7M)
2020-12-11 08:29:09,897 INFO Download rate 33.6M/s  


,lat,lon,plev,ta,time,trajectory_label
obs,,,,,,
0,23.370001,85.330002,45800.0,272.500000,1978-10-01 00:00:00,20000001339
0,23.370001,85.330002,50000.0,272.899994,1978-10-01 00:00:00,20000001339
0,23.370001,85.330002,55200.0,273.200012,1978-10-01 00:00:00,20000001339
0,23.370001,85.330002,60000.0,278.000000,1978-10-01 00:00:00,20000001339
0,23.370001,85.330002,66600.0,282.600006,1978-10-01 00:00:00,20000001339
...,...,...,...,...,...,...
0,23.370001,85.330002,70000.0,281.399994,1978-10-31 12:00:00,20000001381
0,23.370001,85.330002,72800.0,284.000000,1978-10-31 12:00:00,20000001381
0,23.370001,85.330002,80600.0,289.000000,1978-10-31 12:00:00,20000001381


2020-12-11 08:29:11,059 INFO Welcome to the CDS
2020-12-11 08:29:11,060 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2020-12-11 08:29:11,132 INFO Request is queued


...

size - testing:  air_temperature everything


2020-12-11 08:34:44,105 INFO Request is failed
2020-12-11 08:34:44,106 ERROR Message: an internal error occurred processing your request
2020-12-11 08:34:44,106 ERROR Reason:  something went wrong in the data provider service, check your query and try later b'"Request too large - please split"'
2020-12-11 08:34:44,107 ERROR   Traceback (most recent call last):
2020-12-11 08:34:44,107 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 49, in handle_request
2020-12-11 08:34:44,107 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2020-12-11 08:34:44,108 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2020-12-11 08:34:44,108 ERROR       result = proc(context, *context.args, **context.kwargs)
2020-12-11 08:34:44,109 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 118, in __call__
2020-12-11 08:34:44,109 ERROR       return p(*args, **kwargs)
2020-1

an internal error occurred processing your request. something went wrong in the data provider service, check your query and try later b'"Request too large - please split"'.
size - testing:  all variables 1 month all stations


2020-12-11 08:34:45,244 INFO Request is failed
2020-12-11 08:34:45,245 ERROR Message: an internal error occurred processing your request
2020-12-11 08:34:45,245 ERROR Reason:  something went wrong in the data provider service, check your query and try later b'"Request too large - please split"'
2020-12-11 08:34:45,245 ERROR   Traceback (most recent call last):
2020-12-11 08:34:45,246 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/cdshandlers/services/handler.py", line 49, in handle_request
2020-12-11 08:34:45,247 ERROR       result = cached(context.method, proc, context, context.args, context.kwargs)
2020-12-11 08:34:45,247 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/caching.py", line 108, in cached
2020-12-11 08:34:45,248 ERROR       result = proc(context, *context.args, **context.kwargs)
2020-12-11 08:34:45,248 ERROR     File "/opt/cdstoolbox/cdscompute/cdscompute/services.py", line 118, in __call__
2020-12-11 08:34:45,249 ERROR       return p(*args, **kwargs)
2020-1

an internal error occurred processing your request. something went wrong in the data provider service, check your query and try later b'"Request too large - please split"'.
size - testing: finished
